# Load data from databases
This notebook uses the below two prepared databases converted from the raw records.
The output is a flat and de-normalised table.

1 data/accidents_shenzhen.sqlite

2 data/accidents_shenzhen_GIS.sqlite (accidents location by querying Google API)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sqlite3
import pandas as pd
import numpy as np
import os
from pathlib import Path
import json

data_path = os.path.join(Path(os.getcwd()).parent.parent, 'data/')
with open(os.path.join(data_path, 'value_mapping.txt'), encoding='utf-8') as json_file:
    value_dict = json.load(json_file)

## 1 Load data from database for merging and preprocessing

In [3]:
db = "accidents_shenzhen.sqlite"
db_loc = "accidents_shenzhen_GIS.sqlite"

### 1.1 Accident records

In [4]:
## Load accident records
conn = sqlite3.connect(os.path.join(data_path, db))
query = 'SELECT id, acc_id, time, type, weather, reason FROM records'
df_rec = pd.read_sql_query(query, conn)

# Convert acc_id
df_rec.acc_id = df_rec.acc_id.apply(lambda x: int.from_bytes(x, byteorder='little'))
## Preprocess df_rec
df_rec.fillna(value=pd.np.nan, inplace=True)
value_dict_rev = {v: k for k, v in value_dict['acc_type'].items()}
value_dict_rev["滚翻"] = "32"
df_rec.loc[:, 'type'] = df_rec.loc[:, 'type'].apply(lambda x: value_dict_rev[x] if x in value_dict_rev else x)
convert_dict = {'id': pd.Int64Dtype(), 'type': pd.Int64Dtype(), 'reason': pd.Int64Dtype()}
df_rec.loc[:, 'weather'] = df_rec.loc[:, 'weather'].apply(lambda x: int.from_bytes(x, byteorder='little'))
df_rec.loc[:, 'type'] = df_rec.loc[:, 'type'].apply(lambda x: float(x))
df_rec.loc[:, 'reason'] = df_rec.loc[:, 'reason'].apply(lambda x: float(x))
df_rec = df_rec.astype(convert_dict)
df_rec.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  if __name__ == '__main__':


,id,acc_id,time,type,weather,reason
0,1,440398201400141,2014-01-01 00:00:00,11,1,1094
1,2,440309201400034,2014-01-01 00:00:00,11,1,1043
2,3,440310201400029,2014-01-01 00:10:00,11,1,1225
3,4,440391201400001,2014-01-01 00:27:00,11,1,1225
4,5,440391201400051,2014-01-01 00:30:00,35,1,1225


### 1.2 Accident locations

In [5]:
## Load accident locations
conn = sqlite3.connect(os.path.join(data_path, db_loc))
query = 'SELECT rec_id, lat, long FROM gis'
df_loc= pd.read_sql_query(query, conn)
convert_dict = {'rec_id': int, 'lat': float, 'long': float}
df_loc = df_loc.astype(convert_dict)
df_loc.loc[:, 'lat'] = df_loc.loc[:, 'lat'].apply(lambda x: np.nan if x == 999 else x)
df_loc.loc[:, 'long'] = df_loc.loc[:, 'long'].apply(lambda x: np.nan if x == 999 else x)
df_loc.head()

,rec_id,lat,long
0,1,22.593109,114.016152
1,2,22.782450,113.839231
2,3,22.746765,113.884162
3,4,22.626389,113.856753
4,5,22.689902,113.892722


### 1.3 Accident severity

In [6]:
## Load accident severity
conn = sqlite3.connect(os.path.join(data_path, db))
query = 'SELECT rec_id, death, death_s, miss, inj_b, inj_l, inj, loss FROM severity'
df_sev = pd.read_sql_query(query, conn)
for var in ['death', 'death_s', 'miss', 'inj_b', 'inj_l', 'inj', 'loss']:
    df_sev.loc[:, var] = df_sev.loc[:, var].apply(lambda x: int.from_bytes(x, byteorder='little'))
df_sev = df_sev.astype({'rec_id': int})
df_sev.head()

,rec_id,death,death_s,miss,inj_b,inj_l,inj,loss
0,1,0,0,0,0,0,0,500
1,2,0,0,0,0,0,0,3000
2,3,0,0,0,0,1,1,1000
3,4,0,0,0,0,0,0,1200
4,5,0,0,0,0,0,0,1200


### 1.4 geo

In [7]:
## Load geo
conn = sqlite3.connect(os.path.join(data_path, db))
query = 'SELECT rec_id, road_id FROM geo'
df_geo= pd.read_sql_query(query, conn)
df_geo.fillna(value=pd.np.nan, inplace=True)
df_geo = df_geo.astype({'road_id': float})
df_geo = df_geo.astype({'road_id': pd.Int64Dtype()})

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  """


### 1.5 road

In [8]:
## Load road
conn = sqlite3.connect(os.path.join(data_path, db))
query = 'SELECT id, name, type, distr FROM road'
df_road= pd.read_sql_query(query, conn)
df_road.fillna(value=pd.np.nan, inplace=True)
value_dict_rev = {v: k for k, v in value_dict['road_type'].items()}
df_road.loc[:, 'type'] = df_road.loc[:, 'type'].apply(lambda x: value_dict_rev[x] if x in value_dict_rev else x)
convert_dict = {'id': pd.Int64Dtype(), 'type': pd.Int64Dtype(), 'distr': pd.Int64Dtype()}
df_road.loc[:, 'type'] = df_road.loc[:, 'type'].apply(lambda x: float(x))
df_road.loc[:, 'distr'] = df_road.loc[:, 'distr'].apply(lambda x: float(x))
df_road = df_road.astype(convert_dict)

## Join road info to record
df_road = pd.merge(df_geo, df_road, left_on='road_id', right_on='id').drop(columns=['road_id', 'id'])
df_road.rename(columns={'name': 'road_name', 'type': 'road_type'}, inplace=True)
df_road.head()


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  """


,rec_id,road_name,road_type,distr
0,1,福龙快速路,21,440319001
1,84,福龙快速路,21,440319001
2,235,福龙快速路,21,440319001
3,285,福龙快速路,21,440319001
4,639,福龙快速路,21,440319001


### 1.6 People involved in the accidents
This step takes all the crash participants into consideration.

In [9]:
# Read this part of data directly from source files
var_list = ['事故编号', '性别', '年龄', '文化程度', '事故责任', '车辆类型', '交通方式']
df_ppl_list = []
for year in [2014,2015,2016]: #,',2015,2016'
    for selec in ['上半年','下半年']: #,'下半年'
        basicName = '人员信息表'
        xl = pd.ExcelFile(os.path.join(data_path, str(year)+'年'+basicName+'.xls'))
        df_pl = xl.parse(selec)
        df_pl = df_pl.loc[:, var_list]
        df_pl.columns = ['acc_id', 'gender', 'age', 'edu', 'respon', 'veh_type', 'travel_mode']
        df_pl = df_pl.loc[df_pl.acc_id.isin(df_rec.acc_id), :]
        df_ppl_list.append(df_pl)
df_ppl = pd.concat(df_ppl_list)

# Complete veh_type and travel_mode with the other
df_ppl['veh_type'].fillna(df_ppl['travel_mode'], inplace=True)
df_ppl['travel_mode'].fillna(df_ppl['veh_type'], inplace=True)
df_ppl.head()

,acc_id,gender,age,edu,respon,veh_type,travel_mode
0,440301201400335,NaN,NaN,NaN,主要,小型普通客车,小型普通客车
1,440301201400350,女,27.0,NaN,主要,小型普通客车,驾驶小型客车
2,440301201400340,男,24.0,NaN,无责,乘其他非机动车,乘其他非机动车
3,440301201400353,男,33.0,NaN,无责,小型轿车,驾驶小型客车
4,440301201400360,男,6.0,NaN,无责,步行,步行


## 2 Combine data together for further modelling

In [10]:
## Merge location
df = pd.merge(df_rec, df_loc, left_on='id', right_on='rec_id', how="outer").drop(columns=['rec_id'])
print('Merged records and locations: %s' % len(df))

df = pd.merge(df, df_road, left_on='id', right_on='rec_id', how="outer").drop(columns=['rec_id'])
print('Merged road info: %s' % len(df))

df = pd.merge(df, df_sev, left_on='id', right_on='rec_id', how="outer").drop(columns=['rec_id'])
print('Merged severity info: %s' % len(df))

df = pd.merge(df, df_ppl, left_on='acc_id', right_on='acc_id', how="left")
print('Merged people involved: %s' % len(df))

df.drop(columns=['loss'], inplace=True)
print(df.info())

Merged records and locations: 237255
Merged road info: 237255
Merged severity info: 237255
Merged people involved: 436412
<class 'pandas.core.frame.DataFrame'>
Int64Index: 436412 entries, 0 to 436411
Data columns (total 23 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   id           436412 non-null  Int64  
 1   acc_id       436412 non-null  int64  
 2   time         436412 non-null  object 
 3   type         395731 non-null  Int64  
 4   weather      436412 non-null  int64  
 5   reason       432503 non-null  Int64  
 6   lat          431092 non-null  float64
 7   long         431092 non-null  float64
 8   road_name    396124 non-null  object 
 9   road_type    396141 non-null  Int64  
 10  distr        396141 non-null  Int64  
 11  death        436412 non-null  int64  
 12  death_s      436412 non-null  int64  
 13  miss         436412 non-null  int64  
 14  inj_b        436412 non-null  int64  
 15  inj_l        436412 non-null  i

In [11]:
df.head()

,id,acc_id,time,type,weather,reason,lat,long,road_name,road_type,...,miss,inj_b,inj_l,inj,gender,age,edu,respon,veh_type,travel_mode
0,1,440398201400141,2014-01-01 00:00:00,11,1,1094,22.593109,114.016152,福龙快速路,21,...,0,0,0,0,男,46.0,NaN,无责,驾驶小型客车,驾驶小型客车
1,1,440398201400141,2014-01-01 00:00:00,11,1,1094,22.593109,114.016152,福龙快速路,21,...,0,0,0,0,女,47.0,NaN,全部,驾驶大型客车,驾驶大型客车
2,2,440309201400034,2014-01-01 00:00:00,11,1,1043,22.782450,113.839231,107国道,11,...,0,0,0,0,男,62.0,初中,无责,普通正三轮摩托车,驾驶普通摩托车
3,2,440309201400034,2014-01-01 00:00:00,11,1,1043,22.782450,113.839231,107国道,11,...,0,0,0,0,男,30.0,初中,全部,轻型厢式货车,驾驶轻型货车
4,3,440310201400029,2014-01-01 00:10:00,11,1,1225,22.746765,113.884162,根玉路,29,...,0,0,1,1,女,24.0,NaN,同等,电动自行车,电动自行车


In [12]:
df.to_csv(os.path.join(data_path, "data_s1.csv"), index=False)